In [ ]:
import geemap
import ee
import os
import numpy as np

In [ ]:
#ee.Authenticate()

# Define Functions to Retrive, Display, and Save Images

In [ ]:
def RetrieveNAIPimage(lat, long, deltalr = 0.00165, deltaud = 0.00135, date1 = '2018-01-01', date2 = '2020-12-31'):
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date(date1, date2) ).filter(ee.Filter.bounds(region))
    return dataset

def DisplayMap(dataset, lat, long, verbose = True, deltalr = 0.00165, deltaud = 0.00135, zoom = 18):  
    image = dataset.first()
    trueColor = dataset.select(['R', 'G', 'B'])
    trueColorVis = {
      min: 50,
      max: 240,
    }
    if verbose:
        display("images found ",dataset.size())
        print(geemap.image_props(image).getInfo())
        print(geemap.image_stats(image, scale=18).getInfo())
        display(image.geometry().getInfo())
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    regionDebug = ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
    Map = geemap.Map(center=(lat, long), zoom=18)
    Map.addLayer(trueColor, trueColorVis, 'True Color');
    Map.addLayer(region, {'color': 'red'}, '500x500 Image Selected')
    Map.addLayer(regionDebug, {'color': 'green'}, 'NAIP Image Boundary')
    #Map.setOptions('SATELLITE');
    image = dataset.first()

    return Map

def SaveNAIPimage(image ,lat, long, folder, filename, verbose = True, deltalr = 0.00165, deltaud = 0.00135):
    from PIL import Image
    import numpy as np
    fileError.append((folder,filename, lat, long))
    fileTried.append((folder,filename, lat, long, False))
    try:
        region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
        rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

#         rgb_img[:,2] = rgb_img[:,2] 
#         rgb_img[:,1] = rgb_img[:,1] 
       
        if verbose:
            print(rgb_img.shape)
            print(image.getInfo()['properties']['system:footprint']['coordinates'])
        PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
        # Setting the points for cropped image
        left = 9
        top = 0 
        right = 500
        bottom = 500
         
        # Cropped image of above dimension
        # (It will not change original image)
        PIL_image = PIL_image.crop((left, top, right, bottom))
             
        PIL_image.save(f"{folder}/{filename}.png")
        fileError.remove((folder,filename, lat, long))
        fileTried.remove((folder,filename, lat, long, False))
        fileTried.append((folder,filename, lat, long, True))
    except:
        print(f'{(folder,filename, lat, long)}\nThere was a problem:\nLikely the requested image lies outside or overlaps two NAIP images\nOR deltalr and deltaud may be too large for this coordinate - try reducing - the request image is too large' )

In [ ]:
# from PIL import Image 
# im = Image.open("Downloads/Morrison/151swe1.png")
# width, height = im.size
# print(width, height)
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = width
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# im1 = im.crop((left, top, right, bottom))
# im1

# Display Map

Red region is area to be saved

In [ ]:
import geemap
import ee
import os
# If it is not yet authenticated, uncomment the line below
# ee.Authenticate()

ee.Initialize()

long, lat = -106.922, 35.5123  # MM good
long, lat = -109.30146919370144, 40.44057649432536 # DNM


dataset = RetrieveNAIPimage(lat, long)
Map = DisplayMap(dataset, lat, long, verbose = True, zoom = 18)
Map

# Find name of larger maps from NAIP

Yrun Verbosity to True to see more iamge stats:
- near Dinosaur National Monument we find the NAIP image depicted by the green boundary in the map above.
- m_4010938_ne_12_060_20180910. 

I use USGS Earth Explorer to easily grab that map

# Images from NAIP: Public Domain

https://data.nal.usda.gov/dataset/naip-digital-ortho-photo-image-geospatialdatapresentationform-remote-sensing-image

In [ ]:
dataset.getInfo()['properties']['description']

In [ ]:
# Debug
from PIL import Image
import numpy as np

dataset = RetrieveNAIPimage(lat, long)
image = dataset.first()
# folder = 'Downloads/NotMorrison'
# filename = 'bob'
# verbose = False
deltalr = 0.00164
deltaud = 0.00134
region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
# rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# rgb_img[:,2] = rgb_img[:,2] 
# rgb_img[:,1] = rgb_img[:,1] 

# if verbose:
#     print(rgb_img.shape)
#     print(image.getInfo()['properties']['system:footprint']['coordinates'])
# PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = 500
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# PIL_image = PIL_image.crop((left, top, right, bottom))
     
# PIL_image.save(f"{folder}/{filename}.png")

# Save Image

Must reduce dataset down to a single image via filtering and perhaps median aggregation


In [ ]:
# from PIL import Image
# import numpy as np
# fileError = []
# fileTried = []
# dataset = RetrieveNAIPimage(lat, long)
# image = dataset.first()
# folder = 'Downloads/NotMorrison'
# filename = 'bob'
# SaveNAIPimage(image, lat, long, folder, filename, verbose = False)


# Get Coordinates of Region of interest

In [ ]:
#ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
image.getInfo()['properties']['system:footprint']['coordinates']

In [ ]:
import pandas as pd
import time
time.time()
SampleTheseCoords = pd.read_csv('SampleTheseCoordsEdited.csv')
arr = SampleTheseCoords.to_numpy()
N = arr.shape[0]
N

In [ ]:
SampleTheseCoords

# Serial acuisition of images

In [ ]:
# from tqdm import tqdm

# start = time.time()
# print(start)
# for i, row in enumerate(tqdm(arr)):
#     folder = f'Downloads/{row[0]}/'
#     filename = f'{i:03}{row[1]}.png'
#     long = row[2]
#     lat = row[3]
#     dataset = RetrieveNAIPimage(lat, long)
#     image = dataset.first()
#     SaveNAIPimage(image ,lat, long, folder, filename, verbose = False)
# end = time.time()
# print(f'Elapsed: {(end-start)/60:5.1f} min')

# Dask Parallel Acquisition of images

In [ ]:
from tqdm import tqdm
import dask
lazy  = []
fileError = []
fileTried = []

def kernel(row):
    #print(row)
    folder = row[0]
    filename = row[1]
    long = float(row[2])
    lat = float(row[3])
    dataset = RetrieveNAIPimage(lat, long)
    image = dataset.first()
    SaveNAIPimage(image, lat, long, folder, filename, verbose = False)
    lazy.append(row)
    return row
    
ee.Initialize()
start = time.time()
print(start)
for i in range(0,N):
    row = arr[i]
    row[0] = f'data/geemap/{row[0]}/'
    row[1] = f'{i:03}{row[1]}'
    res = dask.delayed(kernel)(row)
    lazy.append(res)
start = time.time()    
dask.compute(*lazy)
end = time.time()
print(f'Elapsed: {(end-start)/60:5.1f} min')

In [ ]:

Files = np.array(fileTried)
Files

In [ ]:
FilesAfterAcquisition = pd.DataFrame(Files, columns=['Folder', 'Filename', 'Latitude', 'Longitude', 'Exists'])
FilesAfterAcquisition

In [ ]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True')].to_csv('FilesAfterAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'False')].to_csv('FilesFailedAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'data/geemap/NotMorrison/')  ]

In [ ]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'data/geemap/Morrison/')  ]

# Populate train & val folders

In [ ]:
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

### Use train test split to partition my list of file names

In [ ]:
# Prep Morrison
df = pd.read_csv('FilesAfterAcquisition.csv')
MorrisonAll = df[df['Folder'] == 'data/geemap/Morrison/']['Filename'].tolist()

Morrison_train, Morrison_val = train_test_split(
    MorrisonAll,  test_size=0.10, random_state=42)

In [ ]:
# Prep NotMorrison
NotMorrisonAll = df[df['Folder'] == 'data/geemap/NotMorrison/']['Filename'].tolist()

NotMorrison_train, NotMorrison_val = train_test_split(
    NotMorrisonAll,  test_size=0.10, random_state=42)

# Moved Train Test split images to associated folders

In [ ]:
import shutil
import os

MT_path = 'data/geemap/train/Morrison/'
MV_path = 'data/geemap/val/Morrison/'
NT_path = 'data/geemap/train/NotMorrison/'
NV_path = 'data/geemap/val/NotMorrison/'

MSrc_path = 'data/geemap/Morrison/'
NSrc_path = 'data/geemap/NotMorrison/'

if not os.path.exists(MT_path):
       os.makedirs(MT_path)
if not os.path.exists(MV_path):
       os.makedirs(MV_path)
if not os.path.exists(NT_path):
       os.makedirs(NT_path)
if not os.path.exists(NV_path):
       os.makedirs(NV_path)
        
for fn in Morrison_train:
    fnm = fn+'.png'
    shutil.copyfile(f'{MSrc_path}{fnm}', f'{MT_path}{fnm}')
for fn in Morrison_val:
    fnm = fn+'.png'
    shutil.copyfile(f'{MSrc_path}{fnm}', f'{MV_path}{fnm}')
        
for fn in NotMorrison_train:
    fnm = fn+'.png'
    shutil.copyfile(f'{NSrc_path}{fnm}', f'{NT_path}{fnm}')
for fn in NotMorrison_val:
    fnm = fn+'.png'
    shutil.copyfile(f'{NSrc_path}{fnm}', f'{NV_path}{fnm}')


In [ ]:
print("Done!")